In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
rdd=sc.textFile("user-ct-test-collection-01.txt") ## it is RDD 
import string
rdd=sc.textFile("user-ct-test-collection-01.txt").map(lambda x: ','.join([''.join(e for e in y if e in string.printable).strip('\"') for y in x.split(',')]))

In [3]:
RDD=rdd.map(lambda x: x.replace("\t",",")).map(lambda x: x.split(","))

In [4]:
TestRDD=RDD.map(lambda t: [t[0],t[1].replace("www.", "").replace("ww.", ""),t[2]\
                           .replace("-", "").replace(" ", "").replace(":", ""),t[3],t[4]])

In [5]:
TestRDD=TestRDD.filter(lambda x: len(x[1])>1)

In [6]:
TestRDD2=TestRDD.filter(lambda t: "AnonID" not in t[0])

In [7]:
TestRDD3=TestRDD2.map(lambda t: [str(t[0]),str(t[1]),int(t[2])/10000,str(t[3]),str(t[4])])

In [8]:
RddNotClickedFiltered=TestRDD3.filter(lambda t: t[3]=="1" or t[3]=="2"or t[3]=="3")


In [9]:
TestRddFinel=RddNotClickedFiltered.map(lambda n: [str(int(n[0])*10000000000+n[2]),n[1],n[3],n[4]])


In [10]:
#--------------------------------------------------------------------------------------

In [11]:
#Count the number of transactions
NumberOfT=TestRddFinel.map(lambda n:  (n[0]))
TransactionIds=NumberOfT.distinct()
NumOfTransactionIds=TransactionIds.count()
NumOfTransactionIds # No. Transaction ID

647932

In [12]:
TidAndSearch=TestRddFinel.map(lambda n: (n[1],n[0])).groupByKey()
#[(X,X->list)()()...]
TidAndSearch=TidAndSearch.filter(lambda x: len(x[1])>10)

In [13]:
TestRddFinel.take(1)

[['1422006032003', 'westchester.gov', '1', 'http://www.westchestergov.com']]

In [14]:
#----------------------------------------------------------------

# -------------------------for explantion----------------------------

TidAndSearchSumT=TidAndSearch.map(lambda row:(row[0],len(row[1])))

TidAndSearchDalution = TidAndSearchSumT.filter(lambda n: n[1]>10)
#Filter the uncommon searches (X, XCount)

SupX = TidAndSearchDalution.map(lambda row: (row[0],float(float(row[1])/10)))

# Sup(x)= Xcount/NumOfTransactionIds

# ---------------------------------------------------------------------

In [15]:
CartSW=TidAndSearch.cartesian(TidAndSearch)# !problem ((x,list),(x,list))

In [16]:
CartSW.take(1)

[(('rheumatoid arthritis',
   <pyspark.resultiterable.ResultIterable at 0x7f6fa18e7290>),
  ('rheumatoid arthritis',
   <pyspark.resultiterable.ResultIterable at 0x7f6fa18d8350>))]

In [17]:
CartSWFilter=CartSW.filter(lambda word: word[0][0] != word[1][0]) # ((x,list),(y,list))
CartSWFilter.take(1)

[(('rheumatoid arthritis',
   <pyspark.resultiterable.ResultIterable at 0x7f6fa18e7090>),
  ('daytona beach cam',
   <pyspark.resultiterable.ResultIterable at 0x7f6fa18ed190>))]

In [18]:
CartSWListIntersection=CartSWFilter\
.map(lambda word: (word[0][0],len(word[0][1]),word[1][0],list(set(word[1][1]) & set(word[0][1])) ) )
# ( X, CountX, Y, COUNT(XUY))
CartSWListIntersection=CartSWListIntersection.filter(lambda x: len(x[3])>1)
CartInListSizeIntersection=CartSWListIntersection.map(lambda x: (x[0],x[1],x[2],len(x[3])))

# -------------------------for explantion----------------------------

XinYCount = CartInListSizeIntersection.map(lambda row: (row[0],row[2],row[3]))

SupXinY = XinYCount.map(lambda row: (row[0],row[1],float(float(row[2])/NumOfTransactionIds)))

#The Dup of (X,Y) and (Y,X) Doesnt effect us

# SUP(XUY)= (X,Y,P(Ex/\Ey))
# P(Ex/\Ey)=> (len(List(X/\Y)))/NumOfTransactionIds

# -------------------------------------------------------------------

In [19]:
#Function CartSWListIntersection [(X, XCount, Y, XUYCount)] => SupXinYComparedToSupX 

In [20]:
ConfidenceXY=CartInListSizeIntersection\
.map(lambda row:(row[0],row[2],float(row[3])/float(row[1])))

ConfidenceXY=CartSWListWithShardTid\
.map(lambda row: 
(row[0],row[2],float(float(float(row[3])/NumOfTransactionIds)/float(float(row[1])/NumOfTransactionIds))))

In [21]:
ConfidenceXYMore06 = ConfidenceXY.filter(lambda row: row[2] > 0.6)
ConfidenceXYMore06.take(10)

[('answers .com', 'find a grave', 0.6226415094339622),
 ('monex', 'cnn', 0.6666666666666666),
 ('monex', 'kitco', 0.9333333333333333),
 ('owens corning stock quote', 'ciena stock quote', 0.6875),
 ('wheaton webmail', 'facebook', 0.7692307692307693),
 ('godwin jewelers bainbridge georgia',
  'bainbridge georgia news',
  0.7142857142857143),
 ('first choice cu', 'ebay', 0.6956521739130435),
 ("ain't it cool", 'dark horizons', 0.6621621621621622),
 ('the birmingham news', 'the decatur daily', 0.6029411764705882),
 ('pierpoint landing', 'free republic', 0.9090909090909091)]

In [22]:
ConfidenceXYMore08 = ConfidenceXYMore06.filter(lambda row: row[2] > 0.8)
ConfidenceXYMore08.take(10)

[('monex', 'kitco', 0.9333333333333333),
 ('pierpoint landing', 'free republic', 0.9090909090909091),
 ('cnn world headlines', 'international herald tribune', 0.9090909090909091),
 ('somerset daily american', 'johnstown tribune-democrat', 0.85),
 ('planters bank', 'suntrust bank', 1.0),
 ('bobhaworth', 'nycsubway', 0.9322033898305084),
 ('indiana gazette', 'johnstown tribune-democrat', 0.8461538461538461),
 ('the richmond times dispatch', 'the news virginian', 0.8095238095238095),
 ('the richmond times dispatch', 'the daily news leader', 0.8690476190476191),
 ('bobhaworth', 'kingstontrioplace', 0.847457627118644)]

In [23]:
ConfidenceXYMore09 = ConfidenceXYMore08.filter(lambda row: row[2] > 0.9)
ConfidenceXYMore09.take(10)

[('monex', 'kitco', 0.9333333333333333),
 ('pierpoint landing', 'free republic', 0.9090909090909091),
 ('cnn world headlines', 'international herald tribune', 0.9090909090909091),
 ('planters bank', 'suntrust bank', 1.0),
 ('bobhaworth', 'nycsubway', 0.9322033898305084),
 ('nancy', 'crankshaft', 0.9090909090909091)]

In [24]:
ConfidenceXYMore06.repartition(1).saveAsTextFile("ConfidenceXYMore06")
ConfidenceXYMore08.repartition(1).saveAsTextFile("ConfidenceXYMore08")
ConfidenceXYMore09.repartition(1).saveAsTextFile("ConfidenceXYMore09")
ConfidenceXY.repartition(1).saveAsTextFile("ConfidenceXY")